# Stratification validation using ICES temperature

In [ ]:
chunk_start

The ability of the model to represent the vertical structure of temperature was assessed by calculating the mixed layer depth (MLD) and comparing it to the MLD calculated from the observations. 




Vertical profiles of temperature were extracted from the ICES oceanographic database. The dataset was pruned to remove profiles with poor quality vertical coverage. First, we removed any profiles without temperature for the top 5 metres. We then removed profiles with less than 10 temperature measurements. Finally, we removed profiles which did not have a temperature record deeper than 80% of the bathymetry. 




## Read in the data

In [ ]:
import seawater as sw
ff = glob.glob("../../matched/point/**/all/temperature/*temperature*.csv")[0]
df = pd.read_csv(ff)
df["pressure"] = sw.eos80.pres(df.depth, df.lat)


## Extract the profiles
This needs more quality control

All it does is extract vertical profiles with more than 9 points

In [ ]:
profiles = (
    df.groupby(["lon", "lat", "day", "month", "year"]).size().reset_index(name = "count")
    .query("count > 9")
    .drop("count", axis = 1)
    .reset_index(drop = True)
)

In [ ]:
def get_mld(i_df):
    try:
        # raise ValueError("here")
        h = HolteAndTalley(list(i_df.pressure),list(i_df.model))
        h1 = h.tempMLD
        max_grad_model = max(h.temp.temperatureGradients)
         ##The temperature algorithms mixed layer depth

        h = HolteAndTalley(list(i_df.pressure),list(i_df.observation))
        h2 = h.tempMLD
        max_grad_obs = max(h.temp.temperatureGradients)
        return  pd.DataFrame({"model": [h1], "observation": h2, "max_grad_model": max_grad_model, "max_grad_obs": max_grad_obs})
    except:
        return  pd.DataFrame({"model": [np.nan], "observation": np.nan, "max_grad_model": np.nan, "max_grad_obs": np.nan})
        return None


profile_mld = (
    profiles
    .merge(df)
    .groupby(["lon", "lat", "year", "month", "day"])
    .apply(get_mld)
    .reset_index()
    .drop(columns = "level_5")
    .loc[:,["lon", "lat", "year", "month", "day", "model", "observation"]]
    .dropna()
)

In [ ]:
%%capture --no-display

%%R -i profile_mld -w 1600 -h 1000
# increase plot size

if (nrow(profile_mld) > 0) {
library(tidyverse, warn.conflicts = FALSE)
world_map <- map_data("world")
profile_mld <- profile_mld 
# get lon, lat limits from profile_mld

xlim = c(min(profile_mld$lon), max(profile_mld$lon))
ylim = c(min(profile_mld$lat), max(profile_mld$lat))



gg <- profile_mld %>%
    ggplot()+
    geom_point(aes(lon, lat))+
    theme_gray(base_size = 28)+
    # add colour scale. Minimum zero, label 100, ">100"
    geom_polygon(data = world_map, aes(long, lat, group = group), fill = "grey60")+
    coord_fixed(xlim = xlim, ylim = ylim, ratio = 1.5) +
    # move legend to the top. Make it 3 cm wide
    theme(legend.position = "bottom", legend.key.width = unit(3, "cm")) +
    # move legend title to the bottom and centre it
    theme(legend.title = element_text(hjust = 1, margin = margin(t = 1, unit = "cm")))

    # move legen

gg
}

In [ ]:
md(f"**Figure {chapter}{i_figure}**: Map of the locations of the temperature profiles used to calculate mixed layer depth.")
i_figure += 1 

In [ ]:
md(f"In total there are **{len(profile_mld)}** profiles across all months. The table below shows the number of profiles per month.") 
import calendar
    
df1 = (
        profile_mld
        .groupby(["month"])
        .size()
        .reset_index(name = "count")
)


gg = (
        ggplot(df1)+
        geom_bar(aes(x = "month", y = "count"), stat = "identity")+
        labs(x = "Month", y = "Number of profiles")+
        scale_x_continuous(breaks = range(1,13), labels = calendar.month_abbr[1:13])

)

gg = gg.draw()
gg




In [ ]:
md(f"**Figure {chapter}{i_figure}**. Number of profiles per month.")
i_figure += 1

In [ ]:
## work out whether model/observations are stratified
# stratification is defined when the difference between the surface and bottom temperature is greater than 0.5 degrees

df_strat = (
    profiles
    .merge(df)
    .merge(profile_mld.loc[:,["lon", "lat", "year", "month", "day"]])
    .drop(columns = "pressure")
    .melt(id_vars = ["lon", "lat", "day", "month", "year", "depth"])
    .rename(columns = {"value": "temperature", "variable": "source"})
    .groupby(["lon", "lat", "day", "month", "year", "source"])
    .agg({"temperature": ["min", "max"]})
    .reset_index()
    .assign(stratified = lambda x: x.temperature["max"] - x.temperature["min"] > 0.5)
    .drop(columns = "temperature")
)
df_strat = df_strat.reset_index(drop = True)
df_strat.columns = [x[0] for x in list(df_strat.columns)]

## Accuracy of stratification

We first identify whether the model and observations are stratified or not. This uses a simple metric of the difference between the surface and bottom temperature. If this is greater than 0.5 degrees, then the profile is considered stratified.



In [ ]:
%%capture --no-display
%%R -i df_strat -w 1600 -h 800

if(nrow(df_strat) > 0){
# increase plot size

library(tidyverse, warn.conflicts = FALSE)
world_map <- map_data("world")
# get lon, lat limits from profile_mld

xlim = c(min(df_strat$lon), max(df_strat$lon))
ylim = c(min(df_strat$lat), max(df_strat$lat))

# cap value at 100

# convert month to Jan, Feb, etc.
# df_strat$month <- factor(df_strat$month, levels = c("Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"))
df_strat <- df_strat %>%
    arrange(month)
df_strat$month <- factor(df_strat$month, levels = df_strat$month, labels = month.abb[df_strat$month])


gg <- df_strat %>%
# # # first six months of the year
    filter(month %in% c("Jan", "Feb", "Mar", "Apr", "May", "Jun")) %>%
    ggplot()+
    geom_point(aes(lon, lat, colour = stratified), size = 0.5)+
    facet_grid(source~month)+
    theme_gray(base_size = 28)+
#     # add colour scale. Minimum zero, label 100, ">100"
    geom_polygon(data = world_map, aes(long, lat, group = group), fill = "grey60")+
    coord_fixed(xlim = xlim, ylim = ylim, ratio = 1.5) +
    # red-blue colour scale
    scale_color_manual(values = c("blue", "red"))+
    # no legend
    theme(legend.position = "none")
#     # move legend to the top. Make it 3 cm wide
#     # move legend title to the bottom and centre it
#     theme(
#     legend.position = "bottom", legend.direction = "horizontal", legend.box = "horizontal", legend.key.width = unit(8.0, "cm"),
#     legend.key.height = unit(1.0, "cm"))
#     # set the legend title to bias


    # move legen

gg

}

In [ ]:
%%capture --no-display
%%R -i df_strat -w 1600 -h 800
# increase plot size
if(nrow(df_strat) > 0){
library(tidyverse, warn.conflicts = FALSE)
world_map <- map_data("world")
# get lon, lat limits from profile_mld

xlim = c(min(df_strat$lon), max(df_strat$lon))
ylim = c(min(df_strat$lat), max(df_strat$lat))

# cap value at 100

# convert month to Jan, Feb, etc.
# df_strat$month <- factor(df_strat$month, levels = c("Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"))
df_strat <- df_strat %>%
    arrange(month)
df_strat$month <- factor(df_strat$month, levels = df_strat$month, labels = month.abb[df_strat$month])


gg <- df_strat %>%
# # # first six months of the year
#final six months of the year
    filter(month %in% c("Jul", "Aug", "Sep", "Oct", "Nov", "Dec")) %>%
    ggplot()+
    geom_point(aes(lon, lat, colour = stratified), size = 0.5)+
    facet_grid(source~month)+
    theme_gray(base_size = 28)+
#     # add colour scale. Minimum zero, label 100, ">100"
    geom_polygon(data = world_map, aes(long, lat, group = group), fill = "grey60")+
    coord_fixed(xlim = xlim, ylim = ylim, ratio = 1.5) +
    # red-blue colour scale
    scale_color_manual(values = c("blue", "red"))+
    # no legend
    theme(legend.position = "none")
#     # move legend to the top. Make it 3 cm wide
#     # move legend title to the bottom and centre it
#     theme(
#     legend.position = "bottom", legend.direction = "horizontal", legend.box = "horizontal", legend.key.width = unit(8.0, "cm"),
#     legend.key.height = unit(1.0, "cm"))
#     # set the legend title to bias


    # move legen

gg

}

In [ ]:
md(f"**Figure {chapter}{i_figure}:** Stratification of the water column. Red indicates stratified, blue indicates unstratified. Stratification is defined based the difference between the surface and bottom temperature being greater than 0.5 degrees.")
i_figure += 1

In [ ]:
gg = (
    ggplot(df_strat.groupby(["source", "month"]).agg({"stratified": "mean"}).reset_index())+
    geom_line(aes("month", "stratified", colour = "source"))+
    labs(x = "Month", y = "Proportion of profiles stratified")
)
gg = gg.draw()
gg


In [ ]:
md(f"**Figure {chapter}{i_figure}**: Proportion of profiles stratified by month in the model and ICES profiles.")
i_figure += 1

## Analysis of mixed layer depth

Mixed layer depth was calculated using the [Holte and Talley (2009) method](https://journals.ametsoc.org/view/journals/atot/26/9/2009jtecho543_1.xml), which uses temperature and pressure as input.


In [ ]:
%%capture --no-display
%%R -i profile_mld -w 1600 -h 700 
# increase plot size

if (nrow(profile_mld) > 0){

library(tidyverse, warn.conflicts = FALSE)
world_map <- map_data("world")
profile_mld <- profile_mld 
# get lon, lat limits from profile_mld

xlim = c(min(profile_mld$lon), max(profile_mld$lon))
ylim = c(min(profile_mld$lat), max(profile_mld$lat))

profile_mld <- profile_mld %>%
    gather(variable, value, model:observation)

# convert month number to month in profile_mld
profile_md <- profile_mld %>%
    arrange(month)
# profile_mld$month <- factor(profile_mld$month, levels = profile_mld$month, labels = month.abb[profile_mld$month])

profile_mld <- profile_mld %>%
    mutate(month = factor(month, levels = 1:12, labels = month.abb[1:12])) %>%
    filter(month %in% c("Jan", "Feb", "Mar", "Apr", "May", "Jun"))

# cap value at 100

profile_mld$value <- ifelse(profile_mld$value > 50, 50, profile_mld$value)

gg <- profile_mld %>%
# first six months of the year
    # filter(month %in% c("Jan", "Feb", "Mar", "Apr", "May", "Jun")) %>% 
    ggplot()+
    geom_point(aes(lon, lat, colour = value))+
    facet_grid(variable~month)+
    theme_gray(base_size = 28)+
    # add colour scale. Minimum zero, label 100, ">100"
    scale_color_viridis_c(breaks = seq(0, 50, 10), labels = c("0", "10", "20", "30", "40", ">50"))+
    geom_polygon(data = world_map, aes(long, lat, group = group), fill = "grey60")+
    coord_fixed(xlim = xlim, ylim = ylim, ratio = 1.5) +
    # remove legend
    theme(legend.position = "none")
    # move legen

gg
}

In [1]:
%%capture --no-display

%%R -i profile_mld -w 1600 -h 800
# increase plot size

if (nrow(profile_mld) > 0){

library(tidyverse, warn.conflicts = FALSE)
world_map <- map_data("world")
profile_mld <- profile_mld 
# get lon, lat limits from profile_mld

xlim = c(min(profile_mld$lon), max(profile_mld$lon))
ylim = c(min(profile_mld$lat), max(profile_mld$lat))

profile_mld <- profile_mld %>%
    gather(variable, value, model:observation)

# convert month number to month in profile_mld
profile_md <- profile_mld %>%
    arrange(month)
# profile_mld$month <- factor(profile_mld$month, levels = profile_mld$month, labels = month.abb[profile_mld$month])

# cap value at 100

profile_mld$value <- ifelse(profile_mld$value > 50, 50, profile_mld$value)

profile_mld <- profile_mld %>%
    mutate(month = factor(month, levels = 1:12, labels = month.abb[1:12])) %>%
    filter(month %in% c("Jan", "Feb", "Mar", "Apr", "May", "Jun")) 

gg <- profile_mld %>%
# final six months of the year
    # filter(month %in% c("Jul", "Aug", "Sep", "Oct", "Nov", "Dec")) %>% 
    ggplot()+
    geom_point(aes(lon, lat, colour = value))+
    facet_grid(variable~month)+
    theme_gray(base_size = 28)+
    # add colour scale. Minimum zero, label 100, ">100"
    geom_polygon(data = world_map, aes(long, lat, group = group), fill = "grey60")+
    coord_fixed(xlim = xlim, ylim = ylim, ratio = 1.5) +
    # move legend to the top. Make it 3 cm wide
    # move legend title to the bottom and centre it
    scale_color_viridis_c(breaks = seq(0, 50, 10), labels = c("0", "10", "20", "30", "40", ">50"),
                       guide = guide_colorbar(title.position = "bottom", title.hjust = 0.5, title.theme = element_text(angle = 0, size = 20, family = "Helvetica"))
  )+
    theme(
    legend.position = "bottom", legend.direction = "horizontal", legend.box = "horizontal", legend.key.width = unit(8.0, "cm"),
    legend.key.height = unit(1.0, "cm"))+
    # set the legend title to bias
    labs(colour = "Mixed Layer Depth (m)")


    # move legen

gg
}

In [ ]:
md(f"**Figure {chapter}{i_figure}**: Spatial distribution of the mixed layer depth (m) in the model and observations.") 
i_figure += 1

In [ ]:
%%capture --no-display

%%R -i profile_mld -w 800 -h 800

library(tidyverse, warn.conflicts = FALSE)

if (nrow(profile_mld) > 0){
    plot_mld <- profile_mld
    plot_mld$month <- factor(plot_mld$month, levels = plot_mld$month, labels = month.abb[plot_mld$month])
    # make the month factors
    plot_mld[["month"]] <- factor(plot_mld[["month"]], levels = month.abb[1:12])    
    gg <- plot_mld %>%
        ggplot()+
        geom_point(aes(model, observation))+
        geom_smooth(aes(model, observation))+
        geom_abline()+
        labs(x = "Model MLD (m)", y = "Observation MLD()")+
        facet_wrap(~month, scales = "free")
        gg
}


In [ ]:
md(f"**Figure {chapter}{i_figure}**: Model versus observation mixed layer depth (m) for each month.")
i_figure += 1

In [ ]:
%%capture --no-display

%%R -i profile_mld -w 2000 -h 2000
# increase plot size
if (nrow(profile_mld) > 0){

library(tidyverse, warn.conflicts = FALSE)
world_map <- map_data("world")
profile_mld <- profile_mld %>%
    mutate(bias = model - observation)
# get lon, lat limits from profile_mld

# cap bia to +/- 100
profile_mld$bias <- ifelse(profile_mld$bias > 25, 25, profile_mld$bias)
profile_mld$bias <- ifelse(profile_mld$bias < -25, -25, profile_mld$bias)

xlim = c(min(profile_mld$lon), max(profile_mld$lon))
ylim = c(min(profile_mld$lat), max(profile_mld$lat))

# convert month number to month in profile_mld
profile_mld <- profile_mld %>%
    arrange(month)
profile_mld$month <- factor(profile_mld$month, levels = profile_mld$month, labels = month.abb[profile_mld$month])

gg <- profile_mld %>%
    ggplot()+
    geom_point(aes(lon, lat, colour = bias))+
    facet_wrap(~month)+
    theme_gray(base_size = 30)+
    scale_color_gradient2(
                       guide = guide_colorbar(title.position = "bottom", title.hjust = 0.5, title.theme = element_text(angle = 0, size = 26, family = "Helvetica"))
  )+
    geom_polygon(data = world_map, aes(long, lat, group = group), fill = "grey60")+
    coord_fixed(xlim = xlim, ylim = ylim, ratio = 1.5) +
    theme(
    legend.position = "bottom", legend.direction = "horizontal", legend.box = "horizontal", legend.key.width = unit(8.0, "cm"),
    legend.key.height = unit(1.0, "cm"))+
    # set the legend title to bias
    labs(colour = "Bias (Model MLD - Observation MLD) (m)")

   

gg
}

In [ ]:
md(f"**Figure {chapter}{i_figure}**: Spatial distribution of the bias (model - observation) in the mixed layer depth (m) in the model and observations.")
i_figure += 1

In [ ]:
chunk_end